In [1]:
import rasterio
import numpy as np

In [2]:
file1 = r'/appl/data/geo/luke/vmi/2021/maaluokka_vmi1x_1721.tif'
file2 = r'/appl/data/geo/luke/vmi/2021/paatyyppi_vmi1x_1721.tif'
file3 = r'/appl/data/geo/luke/vmi/2021/kasvupaikka_vmi1x_1721.tif'

# Read data and metadata from the first file
with rasterio.open(file1) as src:
    maaluokka = src.read(1)
    meta = src.meta
    transform = src.transform
    width = src.width
    height = src.height
    crs = src.crs

# Compute coordinate vectors (center of each pixel along x and y)
x_coords = np.array([transform * (i, 0) for i in range(width)])[:, 0]
y_coords = np.array([transform * (0, j) for j in range(height)])[:, 1]

# Read other rasters
with rasterio.open(file2) as src:
    paatyyppi = src.read(1)

with rasterio.open(file3) as src:
    kasvupaikka = src.read(1)

In [6]:
# maaluokka == 1 & paatyyppi == 1 & kasvupaikka == [1,2,3] --> 1
# maaluokka == 1 & paatyyppi == 1 & kasvupaikka == [4,5,6,7,8,9,10] --> 2

# maaluokka == 1 & paatyyppi == [2,3,4] & kasvupaikka == [1,2,3] --> 3
# maaluokka == 1 & paatyyppi == [2,3,4] & kasvupaikka == [4,5,6,7,8,9,10] --> 4

# maaluokka == 2 & paatyyppi == 1 & kasvupaikka == [1,2,3] --> 5
# maaluokka == 2 & paatyyppi == 1 & kasvupaikka == [4,5,6,7,8,9,10] --> 6

# maaluokka == 2 & paatyyppi == [2,3,4] & kasvupaikka == [1,2,3] --> 7
# maaluokka == 2 & paatyyppi == [2,3,4] & kasvupaikka == [4,5,6,7,8,9,10] --> 8

# maaluokka == 3 & paatyyppi == 1 & kasvupaikka == [1,2,3] --> 9
# maaluokka == 3 & paatyyppi == 1 & kasvupaikka == [4,5,6,7,8,9,10] --> 10

# maaluokka == 3 & paatyyppi == [2,3,4] & kasvupaikka == [1,2,3] --> 11
# maaluokka == 3 & paatyyppi == [2,3,4] & kasvupaikka == [4,5,6,7,8,9,10] --> 12

In [3]:
# Initialize classification array
classification = np.zeros_like(maaluokka, dtype=np.uint8)

# Define categories
kt_ravinteikas = [1,2,3]
kt_ei_ravinteikas = [4,5,6,7,8,9,10]
pt_turvemaa = [2,3,4]

# Apply classification rules
classification[(maaluokka == 1) & (paatyyppi == 1) & np.isin(kasvupaikka, kt_ravinteikas)] = 1
classification[(maaluokka == 1) & (paatyyppi == 1) & np.isin(kasvupaikka, kt_ei_ravinteikas)] = 2
classification[(maaluokka == 1) & np.isin(paatyyppi, pt_turvemaa) & np.isin(kasvupaikka, kt_ravinteikas)] = 3
classification[(maaluokka == 1) & np.isin(paatyyppi, pt_turvemaa) & np.isin(kasvupaikka, kt_ei_ravinteikas)] = 4

classification[(maaluokka == 2) & (paatyyppi == 1) & np.isin(kasvupaikka, kt_ravinteikas)] = 5
classification[(maaluokka == 2) & (paatyyppi == 1) & np.isin(kasvupaikka, kt_ei_ravinteikas)] = 6
classification[(maaluokka == 2) & np.isin(paatyyppi, pt_turvemaa) & np.isin(kasvupaikka, kt_ravinteikas)] = 7
classification[(maaluokka == 2) & np.isin(paatyyppi, pt_turvemaa) & np.isin(kasvupaikka, kt_ei_ravinteikas)] = 8

classification[(maaluokka == 3) & (paatyyppi == 1) & np.isin(kasvupaikka, kt_ravinteikas)] = 9
classification[(maaluokka == 3) & (paatyyppi == 1) & np.isin(kasvupaikka, kt_ei_ravinteikas)] = 10
classification[(maaluokka == 3) & np.isin(paatyyppi, pt_turvemaa) & np.isin(kasvupaikka, kt_ravinteikas)] = 11
classification[(maaluokka == 3) & np.isin(paatyyppi, pt_turvemaa) & np.isin(kasvupaikka, kt_ei_ravinteikas)] = 12

In [4]:
output_file = r'/scratch/project_2000908/nousu/mNFI_to_NEP/classification_16.tif'

with rasterio.open(output_file, "w", **meta) as dst:
    dst.write(classification, 1)